#Analise de séries temporais sobre a contaminação do vírus COVID-19 pelo mundo.

#Importando Bibliotecas


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express  as px
import plotly.graph_objects as go

#Lendo dados

In [ ]:
url = 'https://github.com/neylsoncrepalde/projeto_eda_covid/blob/master/covid_19_data.csv?raw=true'

df = pd.read_csv(url, parse_dates=['ObservationDate', 'Last Update'])
df

#Ajustando Colunas


In [ ]:
#Tipos de cada Coluna
df.dtypes

In [ ]:
#Nomes de colunas não devem ter letras maiúsculas nem caracteres especiais
import re
def corrige_colunas(col_name):
  return re.sub(r"[/| ]", "", col_name).lower()

In [ ]:
#Exemplo
corrige_colunas("AdgE/P ou")

In [ ]:
#corrigindo colunas
df.columns = [corrige_colunas(col) for col in df.columns]
df

#Brasil
Será selecionado apenas os dados do Brasil

In [ ]:
df.loc[df.countryregion == 'Brazil']

,sno,observationdate,provincestate,countryregion,lastupdate,confirmed,deaths,recovered
82,83,2020-01-23,NaN,Brazil,2020-01-23 17:00:00,0.0,0.0,0.0
2455,2456,2020-02-26,NaN,Brazil,2020-02-26 23:53:02,1.0,0.0,0.0
2559,2560,2020-02-27,NaN,Brazil,2020-02-26 23:53:02,1.0,0.0,0.0
2668,2669,2020-02-28,NaN,Brazil,2020-02-26 23:53:02,1.0,0.0,0.0
2776,2777,2020-02-29,NaN,Brazil,2020-02-29 21:03:05,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...
24850,24851,2020-05-15,NaN,Brazil,2020-05-16 02:32:19,220291.0,14962.0,84970.0
25227,25228,2020-05-16,NaN,Brazil,2020-05-17 02:32:32,233511.0,15662.0,89672.0
25604,25605,2020-05-17,NaN,Brazil,2020-05-18 02:32:21,241080.0,16118.0,94122.0
25981,25982,2020-05-18,NaN,Brazil,2020-05-19 02:32:18,255368.0,16853.0,100459.0


In [ ]:
#Para pegar casos que estajam com pelo menos 1 confirmado com COVID
brasil = df.loc[(df.countryregion == 'Brazil') & (df.confirmed > 0)]
brasil

#Casos Confirmados


In [ ]:
#Grafico da evolução de casos confirmados
px.line(brasil, 'observationdate', 'confirmed', title='Casos confirmados no Brasil')

#Novos casos por dia

In [ ]:
brasil['novoscasos'] = list(map(
    lambda x: 0 if (x==0) else brasil['confirmed'].iloc[x] - brasil['confirmed'].iloc[x-1],
    np.arange(brasil.shape[0])
))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#Vizualizando
px.line(brasil, x='observationdate', y='novoscasos', title='Novos casos por dia')

#Mortes

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=brasil.observationdate, y=brasil.deaths, name='Mortes', 
               mode='lines+markers', line=dict(color='red'))
)
#Layout
fig.update_layout(title='Mortes por COVID-19 no Brasil')
fig.show()

#Taxa de Crescimento

In [ ]:
def taxa_crescimento(data, variable, data_inicio=None, data_fim=None):
    # Se data_inicio for None, define como a primeira data disponível no dataset
    if data_inicio == None:
        data_inicio = data.observationdate.loc[data[variable] > 0].min()
    else:
        data_inicio = pd.to_datetime(data_inicio)
        
    if data_fim == None:
        data_fim = data.observationdate.iloc[-1]
    else:
        data_fim = pd.to_datetime(data_fim)
    
    # Define os valores de presente e passado
    passado = data.loc[data.observationdate == data_inicio, variable].values[0]
    presente = data.loc[data.observationdate == data_fim, variable].values[0]
    
    # Define o número de pontos no tempo q vamos avaliar
    n = (data_fim - data_inicio).days
    
    # Calcula a taxa
    taxa = (presente/passado)**(1/n) - 1

    return taxa*100

In [ ]:
#Taxa de crescimento médio do COVID no Brasil em todo o período
taxa_crescimento(brasil, 'confirmed')

16.27183353112116

In [ ]:
def taxa_crescimento_diaria(data, variable, data_inicio=None):
  # Se data_inicio for None, define como a primeira data disponível
  if data_inicio == None:
    data_inicio = data.observationdate.loc[data[variable] > 0].min()
  else:
        data_inicio = pd.to_datetime(data_inicio)
  data_fim = data.observationdate.max()
  # Define o número de pontos no tempo q vamos avaliar
  n = (data_fim - data_inicio).days

  #Taxa calculada de um dia para o outro
  taxas = list(map(
      lambda x: (data[variable].iloc[x] - data[variable].iloc[x-1]) / 
      data[variable].iloc[x-1], range(1, n+1)
  ))
  return np.array(taxas) * 100


In [ ]:
tx_dia = taxa_crescimento_diaria(brasil, 'confirmed')
tx_dia

In [ ]:
primeiro_dia = brasil.observationdate.loc[brasil.confirmed > 0].min()
px.line(x=pd.date_range(primeiro_dia, brasil.observationdate.max())[1:],
        y=tx_dia, title='Taxa de crescimento de casos confirmados no Brasil')

#Predições

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

In [ ]:
confirmados = brasil.confirmed
confirmados.index = brasil.observationdate
confirmados

In [ ]:
res = seasonal_decompose(confirmados)

In [ ]:
fig, (ax1,ax2,ax3,ax4) = plt.subplots(4,1,figsize=(10,8))

ax1.plot(res.observed)
ax2.plot(res.trend)
ax3.plot(res.seasonal)
ax4.plot(confirmados.index, res.resid)
ax4.axhline(0, linestyle='dashed', c='black')
plt.show()

#ARIMA
Media moda integrada autoregressiva

In [ ]:
#isntalando arima
!pip install pmdarima

In [28]:
from pmdarima.arima import auto_arima
modelo = auto_arima(confirmados)

NameError: ignored

In [ ]:
fig = go.Figure(go.Scatter(
    x=confirmados.index, y=confirmados,name='Observados'
))

fig.add_trace(go.Scatter(
    x=confirmados.index,y=modelo.predict_in_sample(),name='Preditos'
))

fig.add_trace(go.Scatter(
    x=pd.date_range('2020-05-20', '2020-06-20',y=modelo.predict(31),name='Forecast')
))

fig.updtade_layout(title='Previsão de casos confirmados no Brasil para os próximos 30 dias')
fig.show()

In [ ]:
pd.date_range('2020-05-01', '2020-05-19')

#Modelo de Crescimento

In [ ]:
#Instalando Biblioteca
!conda install -c conda-forge fbprophet -y

/bin/bash: conda: command not found


In [ ]:
from fbprophet import Prophet

In [ ]:
# preparando os dados
train = confirmados.reset_index()[:-5]
test = confirmados.reset_index()[-5:]

# renomeia colunas
train.rename(columns={"observationdate":"ds","confirmed":"y"},inplace=True)
test.rename(columns={"observationdate":"ds","confirmed":"y"},inplace=True)
test = test.set_index("ds")
test = test['y']

profeta = Prophet(growth="logistic", changepoints=['2020-03-21', '2020-03-30', '2020-04-25', '2020-05-03', '2020-05-10'])

pop = 211463256 
train['cap'] = pop

# Treina o modelo
profeta.fit(train)

# Construindo previsões para o futuro
future_dates = profeta.make_future_dataframe(periods=200)
future_dates['cap'] = pop
forecast =  profeta.predict(future_dates)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=forecast.ds, y=forecast.yhat, name='Predição'))
fig.add_trace(go.Scatter(x=test.index, y=test, name='Observados - Teste'))
fig.add_trace(go.Scatter(x=train.ds, y=train.y, name='Observados - Treino'))
fig.update_layout(title='Predições de casos confirmados no Brasil')
fig.show()